#AAI_6120 Deep Learning & It's Applications

#Farah Malaeb - Project 2: Explainabel AI for Arabic Medical Report Summarization

## Installing the Necessary Libraries

In [2]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.2 MB/s eta 0:00:00


## Importing the Necessary Libraries

In [3]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
import pandas as pd
import os


##Initializing the AraT5v2-base-1024 model and its tokenizer

In [4]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model for the newer version of AraT5
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/araT5v2-base-1024")
model = AutoModelForSeq2SeqLM.from_pretrained("UBC-NLP/araT5v2-base-1024")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

## Loading the Dataset

In [5]:
import pandas as pd
df = pd.read_csv("/content/medical_reports.csv")  # Replace with the actual dataset

print(df.head)

<bound method NDFrame.head of                                        Medical_Report  \
0                 يعاني المريض من صداع شديد منذ يومين   
1   المريضة الحامل في الشهر السادس تعاني من آلام ف...   
2            المريض يعاني من سعال جاف مستمر منذ أسبوع   
3    الشاب يعاني من ألام في أسفل الظهر بعد حادث مروري   
4     المريضة تعاني من صعوبة في التنفس منذ صباح اليوم   
..                                                ...   
61  المريضة تعاني من صعوبة في التنفس مع الشعور بال...   
62              المريض يعاني من ألم في الرأس مع غثيان   
63  المريضة تعاني من احتقان في الأنف وصعوبة في التنفس   
64        المريض يعاني من تهيج في الحلق مع سعال مستمر   
65                 المريضة تعاني من اكتئاب وقلق مستمر   

                              Summary  
0               المريض يعاني من صداع.  
1    المريضة تعاني من آلام في المعدة.  
2           المريض يعاني من سعال جاف.  
3     الشاب يعاني من آلام أسفل الظهر.  
4   المريضة تعاني من صعوبة في التنفس.  
..                                ...  
61  ا

In [6]:
from datasets import Dataset

# Converting the pandas DataFrame to a Hugging Face dataset
#This format is optimized for use with transformer models and enables efficient processing.
dataset = Dataset.from_pandas(df)

# Check the dataset format
print(dataset)

#This conversion is useful for preparing data for training, evaluation, or inference in NLP tasks.

Dataset({
    features: ['Medical_Report', 'Summary'],
    num_rows: 66
})


##Declaring the tokenize function

### tokenize_function(examples): This function is designed to process a batch of examples (medical reports and summaries) for tokenization.

## tokenizer(examples['Medical_Report'], truncation=True, padding='max_length', max_length=512):

- Truncation: Ensures that text longer than 512 tokens is cut off.
- Padding: Uses max_length padding to ensure all sequences have the same length (512 tokens).
- max_length=512: Limits the tokenized sequence to 512 tokens (a common max for transformer models).

## Tokenizing the Targets (Summaries):

- with tokenizer.as_target_tokenizer():
    - This temporarily sets the tokenizer in "target mode" to process the summaries correctly.
- labels = tokenizer(examples['Summary'], truncation=True, padding='max_length', max_length=150):
   - Truncation & Padding: Ensures the summary fits within 150 tokens, making it suitable for training.

In [7]:
# Tokenization function
def tokenize_function(examples):
    # Tokenize the medical_reports (inputs)
    model_inputs = tokenizer(examples['Medical_Report'], truncation=True, padding='max_length', max_length=512)

    # Tokenize the summaries (targets)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['Summary'], truncation=True, padding='max_length', max_length=150)

    # Return the model inputs and labels
    model_inputs['labels'] = labels['input_ids']
    return model_inputs
#This function is essential for preparing the dataset for fine-tuning a sequence-to-sequence model like AraT5 for summarization tasks.

##Applying Tokenization to the Dataset

- Uses the map function to apply tokenize_function to the entire dataset.
- After tokenization, the dataset contains tokenized input IDs, attention masks, and labels.
The original text columns (Medical_Report and Summary) are removed, as the model only needs numerical token representations.

In [8]:
# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove the original columns after tokenization
tokenized_datasets = tokenized_datasets.remove_columns(["Medical_Report", "Summary"])

# Check the tokenized dataset
print(tokenized_datasets)


Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 66
})


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


##Splitting the Dataset into Training and Testing

In [9]:
# Split the dataset into train and validation sets (90% train, 10% validation)
train_dataset, eval_dataset = tokenized_datasets.train_test_split(test_size=0.1).values()


##Defining Training Arguments:

- output_dir='./results' → Saves the training checkpoints and logs.
- evaluation_strategy="epoch" → Evaluates the model at the end of each epoch.
- learning_rate=2e-5 → Sets the learning rate to 0.00002 for fine-tuning.
- per_device_train_batch_size=4 → Uses a small batch size to fit the model into available GPU memory.
- num_train_epochs=3 → Trains the model for 3 epochs.
- weight_decay=0.01 → Applies L2 regularization to prevent overfitting.


In [10]:
# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',  # Output directory for the results
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=2e-5,  # Learning rate
    per_device_train_batch_size=4,  # Batch size for training
    num_train_epochs=3,  # Number of epochs
    weight_decay=0.01,  # Weight decay
)
#This configuration is used to fine-tune AraT5 on medical text summarization while balancing training speed and performance.

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


##First we loading the pretrained model and tokenizer

- Loads AraT5v2-base (a model specialized for Arabic text processing).
- The tokenizer converts text into token IDs.
- The model is ready for sequence-to-sequence learning, ideal for summarization.

##Second we iitialize the trainer
- model=model → Uses the AraT5 model.
- args=training_args → Uses the previously defined training configuration.
- train_dataset=train_dataset → Specifies the training dataset.
- eval_dataset=eval_dataset → Specifies the validation dataset.
- tokenizer=tokenizer → Ensures correct tokenization during training.


In [11]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/araT5v2-base-1024")
model = AutoModelForSeq2SeqLM.from_pretrained("UBC-NLP/araT5v2-base-1024")

# Initialize Trainer with the model, training arguments, and dataset
trainer = Trainer(
    model=model,  # Your pre-trained model
    args=training_args,  # Training arguments
    train_dataset=train_dataset,  # Training dataset
    eval_dataset=eval_dataset,  # Validation dataset
    tokenizer=tokenizer,  # Tokenizer to use
)

#This setup allows the Trainer API (from Hugging Face) to handle fine-tuning the model on medical text summarization efficiently.

<ipython-input-11-f7c1249941b5>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


##Starting the training process

In [12]:
# Start the training process
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,34.172562
2,No log,32.294510
3,No log,31.582533


TrainOutput(global_step=45, training_loss=41.70527615017361, metrics={'train_runtime': 70.2573, 'train_samples_per_second': 2.519, 'train_steps_per_second': 0.641, 'total_flos': 153807899590656.0, 'train_loss': 41.70527615017361, 'epoch': 3.0})

##Evaluating the model

In [13]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Validation Results: {eval_results}")


Validation Results: {'eval_loss': 31.58253288269043, 'eval_runtime': 0.5072, 'eval_samples_per_second': 13.801, 'eval_steps_per_second': 1.972, 'epoch': 3.0}


##Saving the fine-tuned model and tokenizer

In [14]:
# Save the model
model.save_pretrained("./fine_tuned_araT5v2")
tokenizer.save_pretrained("./fine_tuned_araT5v2")


('./fine_tuned_araT5v2/tokenizer_config.json',
 './fine_tuned_araT5v2/special_tokens_map.json',
 './fine_tuned_araT5v2/spiece.model',
 './fine_tuned_araT5v2/added_tokens.json',
 './fine_tuned_araT5v2/tokenizer.json')

In [15]:
!pip install bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

##Importing the necessary libraries for training, evaluation, and performance measurement of a summarization model

In [16]:
from bert_score import score
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
import pandas as pd
import os


##Computing BERTScore

In [17]:
# Compute BERTScore for the model predictions
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode the predicted tokens to text
    predicted_texts = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Calculate BERTScore
    P, R, F1 = score(predicted_texts, label_texts, lang="ar")

    # Return the F1 score as a metric
    return {"bert_score_f1": F1.mean().item()}


##This code takes an input medical report, tokenizes it, sends it to the model for summarization, and then decodes and prints the summary.

In [41]:
import torch

# Define the device
device = "cuda" if torch.cuda.is_available() else "cpu"

test_input = "المريض يعاني من ارتفاع ضغط الدم ومرض السكري من النوع الثاني، مع وجود تاريخ عائلي لأمراض القلب."

# Tokenize input text
test_inputs = tokenizer(test_input, return_tensors="pt", padding=True, truncation=True)

# Move inputs to the correct device
input_ids = test_inputs['input_ids'].to(device)
attention_mask = test_inputs['attention_mask'].to(device)

# Generate the summary
generated_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=250, num_beams=4, early_stopping=True)

# Decode and print the summary
generated_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(f"Generated Summary: {generated_summary}")


Generated Summary: مرض السكري من النوع الثاني وارتفاع ضغط الدم


In [38]:
pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e3ab0ef0f07ceca52d65458d650b5d915b48c2fe7849c9a4150128e006184aec
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


##computing the BLEU score to evaluate the quality of the generated summary

The BLEU score will give an indication of how closely the generated summary aligns with the reference summary. If the generated text is very similar to the reference, the BLEU score will be closer to 1; otherwise, it will be lower

In [43]:
from nltk.translate.bleu_score import sentence_bleu

generated_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
reference_summary = "مرض السكري من النوع الثاني وارتفاع ضغط الدم"

print(f"Generated Summary: {generated_summary}")
print(f"Reference Summary: {reference_summary}")



Generated Summary: مرض السكري من النوع الثاني وارتفاع ضغط الدم
Reference Summary: مرض السكري من النوع الثاني وارتفاع ضغط الدم
